# Train IITM


## Add Google Cloud credentials


---



In [ ]:
import os 
from google.colab import auth
 
auth.authenticate_user()

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "creds.json"

#ensure the path is set correctly
!echo $GOOGLE_APPLICATION_CREDENTIALS

project_id = '<gcloud-project-id>'
bucket_name = 'gs://<bucket-name>'
!gcloud config set project {project_id}

## Clone DeepSpeech v0.7.4 from GitHub

Download corresponding checkpoint also

`/content/model_checkpoints/ ` is where we'll be storing our model checkpoints


In [ ]:
!git clone --branch v0.7.4 https://github.com/mozilla/DeepSpeech

!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.7.4/deepspeech-0.7.4-checkpoint.tar.gz
!tar -xvf deepspeech-0.7.4-checkpoint.tar.gz -C /content/model_checkpoints/ 

## Install DeepSpeech Dependencies

In [ ]:
%cd /content/
!sudo apt-get install python3-venv
!sudo apt-get install python3-dev
!pip install --upgrade pip
!sudo apt-get install sox
!sudo apt-get install sox libsox-fmt-mp3
!sudo apt install git
!pip install librosa==0.7.2
!sudo apt-get install pciutils
!lspci | grep -i nvidia

In [ ]:
!wget https://github.com/git-lfs/git-lfs/releases/download/v2.11.0/git-lfs-linux-amd64-v2.11.0.tar.gz
!tar xvf /content/git-lfs-linux-amd64-v2.11.0.tar.gz -C /content
!sudo bash /content/install.sh
%cd /content/DeepSpeech
!git-lfs pull

In [ ]:
!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.7.4/ds_ctcdecoder-0.7.4-cp36-cp36m-manylinux1_x86_64.whl
!pip install /content/DeepSpeech/ds_ctcdecoder-0.7.4-cp36-cp36m-manylinux1_x86_64.whl

In [ ]:
!pip3 install folium==0.2.1
!pip3 install --upgrade pip==20.0.2 wheel==0.34.2 setuptools==46.1.3
!pip3 install --upgrade --force-reinstall -e .

# Restart Colab after this cell

In [ ]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Sat Jul 11 04:47:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+

## Set default CUDA version

Default CUDA version in Colab is 10.1, need to change to 10.0

In [ ]:
! echo $PATH

import os
os.environ['PATH'] += ":/usr/local/cuda-10.0/bin"
os.environ['CUDADIR'] = "/usr/local/cuda-10.0"
os.environ['LD_LIBRARY_PATH'] = "/usr/lib64-nvidia:/usr/local/cuda-10.0/lib64"

!echo $PATH
!echo $LD_LIBRARY_PATH
!source ~/.bashrc

/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin
/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin:/usr/local/cuda-10.0/bin
/usr/lib64-nvidia:/usr/local/cuda-10.0/lib64


In [ ]:
!env | grep -i cuda

LD_LIBRARY_PATH=/usr/lib64-nvidia:/usr/local/cuda-10.0/lib64
CUDADIR=/usr/local/cuda-10.0
LIBRARY_PATH=/usr/local/cuda/lib64/stubs
CUDA_PKG_VERSION=10-1=10.1.243-1
CUDA_VERSION=10.1.243
NVIDIA_REQUIRE_CUDA=cuda>=10.1 brand=tesla,driver>=384,driver<385 brand=tesla,driver>=396,driver<397 brand=tesla,driver>=410,driver<411
PATH=/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin:/usr/local/cuda-10.0/bin


In [ ]:
%cd /content/
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-get install freeglut3 freeglut3-dev libxi-dev libxmu-dev
!sudo apt-get install build-essential dkms
!sudo dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub

!sudo apt-get update
!sudo apt-get install cuda-10-0

/content
--2020-07-11 04:48:13--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2940 (2.9K) [application/x-deb]
Saving to: ‘cuda-repo-ubuntu1804_10.0.130-1_amd64.deb’

cuda-repo-ubuntu180 100%[===================>]   2.87K  --.-KB/s    in 0s      

2020-07-11 04:48:13 (154 MB/s) - ‘cuda-repo-ubuntu1804_10.0.130-1_amd64.deb’ saved [2940/2940]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libxi-dev is already the newest version (2:1.7.9-1).
libxi-dev set to manually installed.
libxmu-dev is already the newest version (2:1.1.2-2).
libxmu-dev set to manually installed.
freeglut3 is already the newest version (2.8.1-3).


In [ ]:
!sudo rm /usr/local/cuda
!sudo ln -s /usr/local/cuda-10.0 /usr/local/cuda
%ls -l /usr/local/

total 68
drwxr-xr-x  1 root root 4096 Jul 11 04:45 bin/
lrwxrwxrwx  1 root root   20 Jul 11 04:48 cuda -> /usr/local/cuda-10.0/
drwxr-xr-x 16 root root 4096 Jun 26 16:18 cuda-10.0/
drwxr-xr-x  1 root root 4096 Jun 26 16:20 cuda-10.1/
drwxr-xr-x  1 root root 4096 Jun 26 16:27 etc/
drwxr-xr-x  2 root root 4096 Oct 29  2019 games/
drwxr-xr-x  2 root root 4096 Jun 26 16:38 _gcs_config_ops.so/
drwxr-xr-x  1 root root 4096 Jun 26 16:46 include/
drwxr-xr-x  1 root root 4096 Jun 26 16:47 lib/
-rw-r--r--  1 root root 1636 Jun 26 16:40 LICENSE.txt
lrwxrwxrwx  1 root root    9 Oct 29  2019 man -> share/man/
drwxr-xr-x  2 root root 4096 Oct 29  2019 sbin/
-rw-r--r--  1 root root 7291 Jun 26 16:40 setup.cfg
drwxr-xr-x  1 root root 4096 Jun 26 16:27 share/
drwxr-xr-x  2 root root 4096 Oct 29  2019 src/
drwxr-xr-x  2 root root 4096 Jun 26 16:48 xgboost/


In [ ]:
!pip3 uninstall tensorflow
!pip3 install 'tensorflow-gpu==1.15.2'

## If you get a broken-install error while installing, run this

In [ ]:
!sudo apt-get -o Dpkg::Options::="--force-overwrite" install --fix-broken

## Copy data to Colab from Google Bucket

In [ ]:
%cd /content/

!gsutil cp gs://bucket-name/hindi_female_english.zip /content/hindi_female_english.zip
!gsutil cp gs://bucket-name/hindi_male_english.zip /content/hindi_male_english.zip

!unzip hindi_female_english.zip -d /content/hindi_female_english
!unzip hindi_male_english.zip -d /content/hindi_male_english

## Check for extra characters not in alphabet.txt

---

You might need to comment out some lines in `check_characters.py` to get it to work. Run this for all three files as a sanity check before training to avoid future errors.

NOTE: Don't change the original alphabet until you know exactly what you're doing. If you use the original alphabet, you can use the scorer file to improve predictions.

After commenting out some lines, this is how line 44 should look like
```
if not args.disable_unicode_variants:
  unicode_transcript = unicodedata.normalize("NFKC", row[2])
  #if row[2] != unicode_transcript:
    #print("Your input file", in_file, "contains at least one transript with unicode chars on more than one code-point: '{}'. Consider using NFKC normalization: unicodedata.normalize('NFKC', str).".format(row[2]))
    #sys.exit(-1)
all_text |= set(row[2])

```



In [ ]:
!python3 /content/DeepSpeech/training/deepspeech_training/util/check_characters.py -csv /content/train.csv -alpha
!python3 /content/DeepSpeech/training/deepspeech_training/util/check_characters.py -csv /content/dev.csv -alpha
!python3 /content/DeepSpeech/training/deepspeech_training/util/check_characters.py -csv /content/test.csv -alpha

## Training

In [ ]:
%cd /content/DeepSpeech
!python3 DeepSpeech.py --train_cudnn True --early_stop True --es_epochs 6 --n_hidden 2048 --epochs 30 \
  --export_dir /content/models/ --checkpoint_dir /content/model_checkpoints/ \
  --train_files /content/train.csv --dev_files /content/dev.csv --test_files /content/test.csv \
  --learning_rate 0.0001 --train_batch_size 64 --test_batch_size 32 --dev_batch_size 32 --export_file_name 'ft_model' \
  --augment reverb[p=0.2,delay=50.0~30.0,decay=10.0:2.0~1.0] \
  --augment volume[p=0.2,dbfs=-10:-40] \
  --augment pitch[p=0.2,pitch=1~0.2] \
  --augment tempo[p=0.2,factor=1~0.5] 

## Exporting the model for inference

In [ ]:
%cd /content/DeepSpeech/
!python3 util/taskcluster.py --source tensorflow --artifact convert_graphdef_memmapped_format --branch r1.15 --target .
!./convert_graphdef_memmapped_format --in_graph=/content/models/ft_model.pb --out_graph=/content/models/ft_model.pbmm